# SE446 – Week 2B: HDFS Command Line Lab

## 🖥️ Hands-On Terminal Practice with HDFS

---

### 🎯 Learning Objectives

By the end of this lab, you will be able to:

1. **Connect to a remote Hadoop cluster** via SSH
2. **Navigate HDFS** using command-line tools
3. **Create directories and upload files** to HDFS
4. **View and manage files** in the distributed file system
5. **Check cluster status** using admin commands

---

### ⏱️ Estimated Time: 30-45 minutes

---

### 📋 Prerequisites

- Basic knowledge of Linux terminal commands
- SSH credentials (provided by instructor)

---

### 🌐 Our HDFS Cluster

| Component | Value |
|-----------|-------|
| **Master Node** | 134.209.172.50 |
| **Hadoop Version** | 3.4.1 |
| **Total DataNodes** | 2 |
| **Replication Factor** | 2 |
| **Web UI** | https://hdfs.aniskoubaa.org |

---

## 🔧 Part 1: Environment Setup

### 1.1 Install SSH Client (Colab Only)

Google Colab doesn't have `sshpass` installed by default. Let's install it.

In [ ]:
# Install sshpass for password-based SSH (Colab)
!apt-get update -qq && apt-get install -qq sshpass > /dev/null 2>&1
print("✅ SSH tools installed!")

### 1.2 Configure Your Credentials

**⚠️ Important**: Ask your instructor for the SSH password.

#### For Google Colab:
1. Click the 🔑 **Secrets** icon in the left sidebar
2. Add a secret named `HDFS_PASSWORD` with the password
3. Toggle "Notebook access" ON

#### For Local Jupyter:
Create a `.env` file or enter the password when prompted.

In [ ]:
import os

# Cluster connection details
MASTER_HOST = "134.209.172.50"
SSH_USER = "root"
HADOOP_USER = "hadoop"

# Try to load password from Colab secrets or environment
try:
    from google.colab import userdata
    SSH_PASSWORD = userdata.get('HDFS_PASSWORD')
    print("🌐 Running in Google Colab")
    print("✅ Password loaded from Colab Secrets")
except:
    # Try environment variable
    SSH_PASSWORD = os.getenv('HDFS_SSH_PASSWORD')
    if SSH_PASSWORD:
        print("💻 Running locally")
        print("✅ Password loaded from environment")
    else:
        # Manual input (not recommended for shared notebooks)
        from getpass import getpass
        SSH_PASSWORD = getpass("Enter SSH password: ")
        print("✅ Password entered manually")

# Store in environment for shell commands
os.environ['SSHPASS'] = SSH_PASSWORD
os.environ['HDFS_HOST'] = MASTER_HOST

print(f"\n📍 Target: {SSH_USER}@{MASTER_HOST}")

### 1.3 Create SSH Helper Function

This helper makes it easy to run commands on the remote cluster.

In [ ]:
def ssh_run(command, as_hadoop=True):
    """
    Run a command on the remote HDFS cluster via SSH.
    
    Args:
        command: The command to execute
        as_hadoop: If True, run as hadoop user (for HDFS commands)
    """
    if as_hadoop:
        remote_cmd = f"sudo -u hadoop {command}"
    else:
        remote_cmd = command
    
    full_cmd = f'sshpass -e ssh -o StrictHostKeyChecking=no {SSH_USER}@{MASTER_HOST} "{remote_cmd}"'
    return full_cmd

print("✅ SSH helper function ready!")
print("\n📝 Usage: !{ssh_run('hdfs dfs -ls /')}")

### 1.4 Test Connection

In [ ]:
# Test SSH connection
print("🔌 Testing connection to HDFS cluster...")
print("=" * 50)
!{ssh_run('echo "Connection successful! Hostname: $(hostname)"', as_hadoop=False)}
print("\n✅ You are connected to the cluster!")

---

## 📖 Part 2: HDFS Command Basics

### 2.1 Understanding HDFS Commands

All HDFS file operations use the pattern:

```bash
hdfs dfs -<command> [arguments]
```

**Common Commands:**

| Command | Description | Example |
|---------|-------------|----------|
| `-ls` | List files | `hdfs dfs -ls /` |
| `-mkdir` | Create directory | `hdfs dfs -mkdir /mydir` |
| `-put` | Upload file | `hdfs dfs -put file.txt /mydir/` |
| `-get` | Download file | `hdfs dfs -get /mydir/file.txt .` |
| `-cat` | View file content | `hdfs dfs -cat /mydir/file.txt` |
| `-rm` | Delete file | `hdfs dfs -rm /mydir/file.txt` |
| `-du` | Check disk usage | `hdfs dfs -du -h /` |

---

## 🧪 Part 3: Hands-On Exercises

### Exercise 1: Check Hadoop Version

**Task**: Verify the Hadoop installation by checking its version.

In [ ]:
# Check Hadoop version
print("📋 HADOOP VERSION")
print("=" * 40)
!{ssh_run('hadoop version', as_hadoop=False)}

**📝 Question 1**: What version of Hadoop is installed on the cluster?

> Your answer: _________________

---

### Exercise 2: List Root Directory

**Task**: View the contents of the HDFS root directory.

In [ ]:
# List the HDFS root directory
print("📁 HDFS ROOT DIRECTORY (/)")
print("=" * 50)
!{ssh_run('hdfs dfs -ls /')}

**Understanding the Output:**

```
drwxr-xr-x   - hadoop supergroup    0 2024-01-15 10:30 /user
│└┬┘└┬┘└┬┘    └──┬──┘ └────┬────┘  │ └───────┬───────┘ └─┬─┘
│ │  │  │        │        │       │         │            │
│ │  │  │        │        │       │         │            └─ Path
│ │  │  │        │        │       │         └─ Modification time
│ │  │  │        │        │       └─ Size (0 for directories)
│ │  │  │        │        └─ Group
│ │  │  │        └─ Owner
│ │  │  └─ Execute permission
│ │  └─ Write permission
│ └─ Read permission
└─ d = directory, - = file
```

---

### Exercise 3: Check Cluster Status

**Task**: Get a report of the HDFS cluster health.

In [ ]:
# Get HDFS cluster report
print("📊 HDFS CLUSTER REPORT")
print("=" * 60)
!{ssh_run('hdfs dfsadmin -report')}

**📝 Question 2**: Fill in the table based on the cluster report:

| Metric | Value |
|--------|-------|
| Live DataNodes | _____ |
| Total Capacity | _____ GB |
| DFS Used | _____ % |
| DFS Remaining | _____ GB |

---

### Exercise 4: Create Your Directory

**Task**: Create a personal directory in HDFS.

**⚠️ Replace `YOUR_NAME` with your actual name (no spaces)!**

In [ ]:
# ============================================
# 👇 CHANGE THIS TO YOUR NAME (no spaces!) 👇
# ============================================
STUDENT_NAME = "student_demo"
# ============================================

print(f"📁 Creating directory for: {STUDENT_NAME}")
print("=" * 50)

# Create the directory
!{ssh_run(f'hdfs dfs -mkdir -p /students/{STUDENT_NAME}')}
print(f"✅ Created: /students/{STUDENT_NAME}")

# Verify
print("\n📋 Verifying directory exists:")
!{ssh_run('hdfs dfs -ls /students')}

---

### Exercise 5: Create and Upload a File

**Task**: Create a text file and upload it to HDFS.

In [ ]:
# Step 1: Create a sample file on the server
print("📝 STEP 1: Creating a sample file")
print("=" * 50)

sample_content = f"Hello from {STUDENT_NAME}!\\nThis is my first HDFS file.\\nCreated during SE446 Lab."
!{ssh_run(f'echo -e "{sample_content}" > /tmp/hello_{STUDENT_NAME}.txt', as_hadoop=False)}

# Verify local file
print("Local file content:")
!{ssh_run(f'cat /tmp/hello_{STUDENT_NAME}.txt', as_hadoop=False)}

In [ ]:
# Step 2: Upload to HDFS
print("\n📤 STEP 2: Uploading to HDFS")
print("=" * 50)

!{ssh_run(f'hdfs dfs -put /tmp/hello_{STUDENT_NAME}.txt /students/{STUDENT_NAME}/')}
print("✅ File uploaded!")

# Verify upload
print(f"\n📋 Files in /students/{STUDENT_NAME}/:")
!{ssh_run(f'hdfs dfs -ls /students/{STUDENT_NAME}/')}

---

### Exercise 6: View File Content

**Task**: Read your file directly from HDFS.

In [ ]:
# View file content using -cat
print("📄 FILE CONTENT FROM HDFS")
print("=" * 50)
!{ssh_run(f'hdfs dfs -cat /students/{STUDENT_NAME}/hello_{STUDENT_NAME}.txt')}

---

### Exercise 7: Check Disk Usage

**Task**: Check how much space your files are using.

In [ ]:
# Check disk usage
print("📊 DISK USAGE")
print("=" * 50)

# Your directory
print(f"\n📁 Your directory (/students/{STUDENT_NAME}):")
!{ssh_run(f'hdfs dfs -du -h /students/{STUDENT_NAME}')}

# Overall HDFS usage
print("\n📁 Overall HDFS usage:")
!{ssh_run('hdfs dfs -df -h')}

---

### Exercise 8: File Replication Info

**Task**: Check the replication factor and block locations of your file.

In [ ]:
# Check file statistics
print("📊 FILE STATISTICS")
print("=" * 50)

# Get replication factor
print("\n🔢 Replication factor:")
!{ssh_run(f"hdfs dfs -stat '%r' /students/{STUDENT_NAME}/hello_{STUDENT_NAME}.txt")}

# Get block size
print("\n📦 Block size:")
!{ssh_run(f"hdfs dfs -stat '%o' /students/{STUDENT_NAME}/hello_{STUDENT_NAME}.txt")}

# Get file size
print("\n📏 File size (bytes):")
!{ssh_run(f"hdfs dfs -stat '%b' /students/{STUDENT_NAME}/hello_{STUDENT_NAME}.txt")}

In [ ]:
# Check which DataNodes store your file
print("📍 BLOCK LOCATIONS")
print("=" * 50)
print("\nWhich DataNodes have copies of your file?")
!{ssh_run(f'hdfs fsck /students/{STUDENT_NAME}/hello_{STUDENT_NAME}.txt -files -blocks -locations')}

**📝 Question 3**: On how many DataNodes is your file stored?

> Your answer: _________________

**📝 Question 4**: Why is the file replicated to multiple nodes?

> Your answer: _________________

---

### Exercise 9: Copy Files Within HDFS

**Task**: Create a backup copy of your file within HDFS.

In [ ]:
# Copy file within HDFS
print("📋 COPYING FILE WITHIN HDFS")
print("=" * 50)

# Create backup directory
!{ssh_run(f'hdfs dfs -mkdir -p /students/{STUDENT_NAME}/backup')}

# Copy file
!{ssh_run(f'hdfs dfs -cp /students/{STUDENT_NAME}/hello_{STUDENT_NAME}.txt /students/{STUDENT_NAME}/backup/')}
print("✅ File copied!")

# Verify
print(f"\n📋 All files in /students/{STUDENT_NAME}/:")
!{ssh_run(f'hdfs dfs -ls -R /students/{STUDENT_NAME}/')}

---

### Exercise 10: Download File from HDFS

**Task**: Download your file from HDFS to the local server.

In [ ]:
# Download file from HDFS
print("📥 DOWNLOADING FROM HDFS")
print("=" * 50)

# Download using -get
!{ssh_run(f'hdfs dfs -get /students/{STUDENT_NAME}/hello_{STUDENT_NAME}.txt /tmp/downloaded_{STUDENT_NAME}.txt')}
print("✅ File downloaded!")

# Verify download
print("\n📄 Downloaded file content:")
!{ssh_run(f'cat /tmp/downloaded_{STUDENT_NAME}.txt', as_hadoop=False)}

---

### Exercise 11: Safe Mode Check

**Task**: Check if the NameNode is in safe mode.

In [ ]:
# Check safe mode status
print("🔒 SAFE MODE STATUS")
print("=" * 50)
!{ssh_run('hdfs dfsadmin -safemode get')}

print("\n💡 What is Safe Mode?")
print("   └─ During safe mode, HDFS is read-only")
print("   └─ This happens during startup or maintenance")
print("   └─ Normal status: 'Safe mode is OFF'")

---

### Exercise 12: Create Sample Data for MapReduce

**Task**: Prepare data that we'll use in the next lecture on MapReduce.

In [ ]:
# Create sample word count data
print("📝 CREATING SAMPLE DATA FOR MAPREDUCE")
print("=" * 50)

# Create sample text file
wordcount_data = """Hello World
Hello Hadoop
Hello HDFS
Hadoop is great
HDFS stores data
MapReduce processes data
Hello Big Data
Big Data is the future"""

# Write to remote server
!{ssh_run(f'echo "{wordcount_data}" > /tmp/wordcount_input.txt', as_hadoop=False)}

# Upload to HDFS
!{ssh_run(f'hdfs dfs -mkdir -p /students/{STUDENT_NAME}/mapreduce')}
!{ssh_run(f'hdfs dfs -put /tmp/wordcount_input.txt /students/{STUDENT_NAME}/mapreduce/')}

print("✅ Sample data created!")
print(f"\n📋 File: /students/{STUDENT_NAME}/mapreduce/wordcount_input.txt")
print("\n📄 Content:")
!{ssh_run(f'hdfs dfs -cat /students/{STUDENT_NAME}/mapreduce/wordcount_input.txt')}

---

## 📚 Part 4: Command Reference

### Quick Reference Card

```bash
# ═══════════════════════════════════════════════════════════════
# HDFS COMMAND QUICK REFERENCE
# ═══════════════════════════════════════════════════════════════

# --- File Navigation ---
hdfs dfs -ls /path              # List directory
hdfs dfs -ls -R /path           # List recursively
hdfs dfs -ls -h /path           # Human-readable sizes

# --- Directory Operations ---
hdfs dfs -mkdir /path           # Create directory
hdfs dfs -mkdir -p /path/sub    # Create with parents

# --- File Operations ---
hdfs dfs -put local remote      # Upload file
hdfs dfs -get remote local      # Download file
hdfs dfs -cat /path/file        # View content
hdfs dfs -head /path/file       # First 1KB
hdfs dfs -tail /path/file       # Last 1KB
hdfs dfs -cp src dst            # Copy within HDFS
hdfs dfs -mv src dst            # Move/rename

# --- Delete ---
hdfs dfs -rm /path/file         # Delete file
hdfs dfs -rm -r /path/dir       # Delete directory
hdfs dfs -rm -skipTrash /file   # Delete permanently

# --- Information ---
hdfs dfs -du -h /path           # Disk usage
hdfs dfs -df -h                 # Free space
hdfs dfs -stat '%r %o %b' /f    # File stats
hdfs dfs -count /path           # Count files/dirs

# --- Admin Commands ---
hdfs dfsadmin -report           # Cluster report
hdfs dfsadmin -safemode get     # Safe mode status
hdfs fsck /path                 # File system check
# ═══════════════════════════════════════════════════════════════
```

---

## 🧹 Part 5: Cleanup (Optional)

Optionally clean up your test files.

In [ ]:
# List all your files before cleanup
print("📋 YOUR FILES IN HDFS")
print("=" * 50)
!{ssh_run(f'hdfs dfs -ls -R /students/{STUDENT_NAME}/')}

# Uncomment to delete (be careful!)
# !{ssh_run(f'hdfs dfs -rm -r /students/{STUDENT_NAME}')}
# print("\n🗑️ Directory deleted")

---

## ✅ Lab Checklist

Before submitting, make sure you completed:

- [ ] Connected to the HDFS cluster via SSH
- [ ] Checked Hadoop version
- [ ] Listed root directory contents
- [ ] Viewed cluster status report
- [ ] Created your personal directory
- [ ] Uploaded a file to HDFS
- [ ] Viewed file content with `-cat`
- [ ] Checked file replication and block locations
- [ ] Copied files within HDFS
- [ ] Downloaded a file from HDFS
- [ ] Created sample data for MapReduce lab
- [ ] Answered all questions

---

## 📝 Lab Submission

### Your Answers

Copy and fill in your answers below:

```
Student Name: ________________________
Student ID:   ________________________

Question 1 (Hadoop Version): ________________________

Question 2 (Cluster Status):
  - Live DataNodes: ________________________
  - Total Capacity: ________________________
  - DFS Used %:     ________________________
  - DFS Remaining:  ________________________

Question 3 (Replication): ________________________

Question 4 (Why replication?): 
________________________
________________________
________________________
```

---

## 📚 Summary

In this lab, you learned:

| Skill | Commands Used |
|-------|---------------|
| **Connect** to cluster | `ssh user@host` |
| **List** files | `hdfs dfs -ls` |
| **Create** directories | `hdfs dfs -mkdir` |
| **Upload** files | `hdfs dfs -put` |
| **Download** files | `hdfs dfs -get` |
| **View** content | `hdfs dfs -cat` |
| **Check** status | `hdfs dfsadmin -report` |
| **Analyze** blocks | `hdfs fsck -files -blocks` |

---

## 🎯 Next Steps

1. **Practice**: Try more commands from the reference card
2. **Explore**: Visit the Web UI at https://hdfs.aniskoubaa.org
3. **Prepare**: Next lecture covers MapReduce!

---

**🔗 Resources:**
- [HDFS Commands Guide](https://hadoop.apache.org/docs/r3.4.1/hadoop-project-dist/hadoop-common/FileSystemShell.html)
- [HDFS Admin Commands](https://hadoop.apache.org/docs/r3.4.1/hadoop-project-dist/hadoop-hdfs/HDFSCommands.html)
- Cluster Web UI: https://hdfs.aniskoubaa.org